<a href="https://colab.research.google.com/github/abasilio91/doctorship/blob/main/fcp_nara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import differential_evolution as de
from scipy.optimize import fsolve
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
!git clone https://github.com/Narabcs/doctorship/

Cloning into 'doctorship'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 67 (delta 30), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (67/67), done.


### Etapa 01:

Calculando os valores de tortuosidade utilizando os dados da água
---

In [ ]:
dados = pd.read_csv('/content/doctorship/parameters_water.csv')
dados['y_ress'] = dados['yexp']/dados['xexp']
dados.head()

,yexp,mi,xexp,pf,meio,y_ress
0,31500000.0,0.000891,0.0080,997.05,50,3.937500e+09
1,63000000.0,0.000891,0.0125,997.05,50,5.040000e+09
2,94500000.0,0.000891,0.0148,997.05,50,6.385135e+09
3,126000000.0,0.000891,0.0173,997.05,50,7.283237e+09
4,157000000.0,0.000891,0.0193,997.05,50,8.134715e+09


In [ ]:
ress_50 = stats.linregress(x=dados.query('meio==50')['xexp'],y=dados.query('meio==50')['y_ress'])
ress_120 = stats.linregress(x=dados.query('meio==120')['xexp'],y=dados.query('meio==120')['y_ress'])

results = {'meio=50':{'r_squared':ress_50.rvalue,
           'intercept': ress_50.intercept,
           'slope': ress_50.slope},
           'meio=120':{'r_squared':ress_120.rvalue,
           'intercept': ress_120.intercept,
           'slope': ress_120.slope}
           }

results = pd.DataFrame(results)
results

,meio=50,meio=120
r_squared,9.905125e-01,9.960379e-01
intercept,1.680439e+08,2.275211e+08
slope,4.229806e+11,2.715969e+11


In [ ]:
def tortuosity(slope,k,pf):
  c = slope*k**0.5/pf

  return c

In [ ]:
c_50 = tortuosity(results['meio=50'].slope,1.48e-11,dados['pf'].unique())
c_120 = tortuosity(results['meio=120'].slope,3.95e-11,dados['pf'].unique())
print(f'c_50 = {float(c_50)}, c_120 = {float(c_120)}')

c_50 = 1632.0532436779447, c_120 = 1712.010470506321


### Etapa 02: 

Estimando os valores de K para as equações de Forchheimmer e Darcy.

In [ ]:
dados_estimacao = pd.read_csv('/content/doctorship/parameters_onlygx.csv')
dados_estimacao.head()

,yexp,m,n,xexp,es,pf,gx,meio,c
0,31500000.0,0.774,0.326,0.00306,0.354,998.28,0.2,50,1650.0
1,63000000.0,0.784,0.321,0.00703,0.354,998.28,0.2,50,1650.0
2,94500000.0,0.762,0.325,0.01030,0.354,998.28,0.2,50,1650.0
3,126000000.0,0.767,0.322,0.01180,0.354,998.28,0.2,50,1650.0
4,157000000.0,0.771,0.317,0.01450,0.354,998.28,0.2,50,1650.0


In [ ]:
def forch_opt(x, dados):
  forch = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/x[0]**0.5)**(dados[2]-1))/x[0] * dados[3] + dados[6]*dados[5]/x[0]**0.5 * dados[3]**2
  err_rel = abs(dados[0] - forch) / dados[0] 

  return err_rel

def darcy_opt(k, dados):
  darcy = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/k**0.5)**(dados[2]-1))/k * dados[3]
  err_rel = abs(dados[0] - darcy) / dados[0] 
  return err_rel

def modelo(dados, model:str='forch'):
  
  if model == 'forch':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_forch']**0.5)**(dados['n']-1))
    y = mu/dados['k_forch'] * dados['xexp'] + dados['c']*dados['pf']/dados['k_forch']**0.5 * dados['xexp']**2

  if model == 'darcy':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_darcy']**0.5)**(dados['n']-1))
    y = mu/dados['k_darcy'] * dados['xexp']

  return y, mu

In [ ]:
params_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=forch_opt,bounds=params_bound,args=(dados_selecionados,))
  results.append(value)

df_forch = pd.DataFrame(results)

k_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=darcy_opt,bounds=k_bound,args=(dados_selecionados,))
  results.append(value)

df_darcy = pd.DataFrame(results)

In [ ]:
k_forch = []
k_darcy = []

for item in zip(df_forch.x,df_darcy.x):
  k_forch.append(item[0][0])
  k_darcy.append(item[1][0])
dados_estimacao['k_forch'] = k_forch
dados_estimacao['k_darcy'] = k_darcy
dados_estimacao['dpl_forch'], dados_estimacao['mu_forch'] = modelo(dados_estimacao,model='forch')
dados_estimacao['dpl_darcy'], dados_estimacao['mu_darcy'] = modelo(dados_estimacao,model='darcy')
dados_estimacao.head()

,yexp,m,n,xexp,es,pf,gx,meio,c,k_forch,k_darcy,dpl_forch,mu_forch,dpl_darcy,mu_darcy
0,31500000.0,0.774,0.326,0.00306,0.354,998.28,0.2,50,1650.0,6.616216e-13,1.102050e-13,31500000.0,0.002711,4.114602e+07,0.001482
1,63000000.0,0.784,0.321,0.00703,0.354,998.28,0.2,50,1650.0,2.138180e-12,8.233105e-14,63000000.0,0.002229,6.300000e+07,0.000738
2,94500000.0,0.762,0.325,0.01030,0.354,998.28,0.2,50,1650.0,3.867145e-12,5.487746e-14,94500000.0,0.002117,9.450000e+07,0.000503
3,126000000.0,0.767,0.322,0.01180,0.354,998.28,0.2,50,1650.0,3.652774e-12,3.648027e-14,126000000.0,0.001857,1.260000e+08,0.000390
4,157000000.0,0.771,0.317,0.01450,0.354,998.28,0.2,50,1650.0,5.183517e-12,2.670566e-14,157000000.0,0.001748,1.570000e+08,0.000289


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

dados_estimacao.to_csv('/gdrive/MyDrive/Colab Notebooks/etapa_3.csv')

### Etapa 03:

Estimar a permeabilidade da torta

In [23]:
def fo_eval(x,params,num_points):

  def diff_model(x,params,beta,num_points):
    # beta = [bkc, bkm]

    dP = params['dP']
    m = params['m']
    n = params['n']
    ec = params['ec']
    pfil = params['pfil']
    ps = params['ps']
    A = params['A']
    Cs = params['Cs']
    lm = params['lm']
    km0 = params['km0']
    t = params['t']

    h = t/(num_points-1)
    kc = np.exp(-beta[0]/t) - 1
    km = km0*np.exp(-beta[1]*t)
    alpha = 1 / (ps*(1-ec)*kc)
    B = dP*A**n/(m*(1.1/(2.5*ec)**0.5*1/(kc**0.5))**(n-1))
    omega = Cs*pfil/(A*ps*(1-ec)*kc)

    x = np.zeros(num_points)
    x[0] = 0
    x[-1] = 2

    dx = np.zeros_like(x)
    dx[0] = (x[1] - x[0])/(2*h)
    dx[-1] = (x[-1] - x[-2])/(2*h)
    dx[1:-1] = (x[2:] - x[:-2])/(2*h)
    
    return dx**n*(omega*x + lm/km) - B

  beta = [x[0], x[1]]
  V_cal = []
  
  for row in params.iterrows():
    exp_line = dict(row[1])
    guess = np.zeros(num_points, float)
    sol = fsolve(func=diff_model, x0=guess, args=(exp_line,beta,num_points))
    V_cal.append(sol[-1])

  params['V_cal'] = V_cal

  rel_err_for_row = abs(params['V']-params['V_cal']) / params['V']
  mean_rel_err = rel_err_for_row.mean()

  return mean_rel_err

In [6]:
dados_torta = pd.read_csv('/content/doctorship/parameters_gx_caco3.csv')
dados_torta.head()

,dP,m,n,ec,pfil,ps,A,Cs,t,V,lm,km0,XG,medium
0,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,60,0.000003,0.00635,1.480000e-11,0.2,50
1,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,300,0.000008,0.00635,1.480000e-11,0.2,50
2,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,600,0.000011,0.00635,1.480000e-11,0.2,50
3,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,900,0.000013,0.00635,1.480000e-11,0.2,50
4,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,1200,0.000014,0.00635,1.480000e-11,0.2,50


In [7]:
data_selection = dados_torta.query('XG==0.2 and medium==50 and dP==2e5')
data_selection

,dP,m,n,ec,pfil,ps,A,Cs,t,V,lm,km0,XG,medium
0,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,60,0.000003,0.00635,1.480000e-11,0.2,50
1,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,300,0.000008,0.00635,1.480000e-11,0.2,50
2,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,600,0.000011,0.00635,1.480000e-11,0.2,50
3,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,900,0.000013,0.00635,1.480000e-11,0.2,50
4,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,1200,0.000014,0.00635,1.480000e-11,0.2,50
5,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,1500,0.000016,0.00635,1.480000e-11,0.2,50
6,200000.0,0.81,0.3,0.51,1080.0,2808,0.00229,0.000133,1800,0.000017,0.00635,1.480000e-11,0.2,50


In [24]:
params_bound = [(-100,100), (-100,100)]
# params_bound = [bkc, bkm]
# easter egg
results = []
num_points = 20
value = de(func=fo_eval, bounds=params_bound, args=(data_selection,num_points), maxiter=5000)
results.append(value)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher

In [25]:
df_torta = pd.DataFrame(results)
df_torta

,x,fun,nfev,nit,message,success
0,"[-26.072082086746708, -33.63394472701329]",1.0,63,1,Optimization terminated successfully.,True


In [ ]:
k_torta = []

for item in df_torta.x:
  k_torta.append(float(item))
  
dados_torta['k_torta'] = k_torta
dados_torta.head()